In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
import os
with zipfile.ZipFile('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip','r') as zip_ref:
    zip_ref.extractall("./sentiment-analysis-on-movie-reviews/")
with zipfile.ZipFile('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip','r') as zip_ref:
    zip_ref.extractall("./sentiment-analysis-on-movie-reviews/")


In [ ]:
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [ ]:
data=pd.read_table("/kaggle/working/sentiment-analysis-on-movie-reviews/train.tsv",sep='\t')
data=data[['Phrase','Sentiment']].copy()

In [ ]:
dff=[len(i.split(" ")) for i in data.Phrase[:10]]
max(dff)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.Sentiment.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=data.Sentiment)

data['data_type'] = ['not_set']*data.shape[0]

data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'


In [ ]:

# Name of the BERT model to use
model_name = 'bert-base-cased'

# Max length of tokens
max_length = max(dff)+3

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)



In [ ]:
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

bert = TFAutoModel.from_pretrained('bert-base-cased')
embeddings = bert.bert(inputs)[1]  # access pooled activations with [1]

# convert bert embeddings into 5 output classes
x =Dense(1024, activation='relu')(embeddings)
y =Dense(5, activation='softmax', name='outputs')(x)

model = Model(inputs=inputs, outputs=y)
#model.layers[2].trainable = False

# Take a look at the model
model.summary()

In [ ]:
y_senti = to_categorical(data[data.data_type=='train'].Sentiment)

# Tokenize the input 
x = tokenizer(
    text=data[data.data_type=='train'].Phrase.to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

train=tf.data.Dataset.from_tensor_slices((x['input_ids'], x['attention_mask'], y_senti))
def map_func(input_ids, masks, labels):
    # convert three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

train = train.map(map_func)
batch_size = 32

# shuffle and batch - dropping any remaining samples that don't cleanly
train = train.shuffle(100).batch(batch_size, drop_remainder=True)

In [ ]:
train.take(1)

In [ ]:
y_senti = to_categorical(data[data.data_type=='val'].Sentiment)

# Tokenize the input 
x = tokenizer(
    text=data[data.data_type=='val'].Phrase.to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

val = tf.data.Dataset.from_tensor_slices((x['input_ids'], x['attention_mask'], y_senti))
val = val.map(map_func)
val = val.shuffle(100).batch(batch_size, drop_remainder=True)

In [ ]:
optimizer = Adam(lr=1e-5, decay=1e-6)
loss = CategoricalCrossentropy()
acc = CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
# Fit the model
history = model.fit(
    train,
    validation_data=val,
    epochs=3)

In [ ]:
model.save('./sentiment-analysis-on-movie-reviews/sentiment_model')

In [ ]:
#model = tf.keras.models.load_model('./sentiment-analysis-on-movie-reviews/sentiment_model')
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
test=pd.read_table("/kaggle/working/sentiment-analysis-on-movie-reviews/test.tsv",sep='\t')
#data=data[['Phrase','Sentiment']].copy()

In [ ]:
test.shape[0]

In [ ]:
x = tokenizer(
    text=test.Phrase.to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
x

In [ ]:
items=tf.data.Dataset.from_tensor_slices((x['input_ids'],x['attention_mask']))

In [ ]:
items

In [ ]:
def map_func(input_ids, masks):
    return {'input_ids': input_ids, 'attention_mask': masks}

items = items.map(map_func)
items = items.batch(32)

In [ ]:
items

In [ ]:
predictions=model.predict(items).argmax(axis=-1)

In [ ]:
len(predictions)

In [ ]:
predictions

In [ ]:
submission = pd.DataFrame()
submission['PhraseId'] = test['PhraseId']
submission['Sentiment'] = predictions
submission.to_csv("submission.csv", index=False)